In [3]:
import pandas as pd
import numpy as np
import datetime as dt
import warnings
import glob
import os

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None
pd.options.display.float_format ='{:,.3f}'.format

In [4]:
current_date = dt.date.today()
day = current_date.day - 1 # данные по продажам мы получаем за предыдущий день
month = current_date.month
year = current_date.year

In [11]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Выгрузки из DWH\Продажи по КБК из DWH new\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

df_dwh = pd.read_excel(latest_file)

In [42]:
# мы смотрим продажи со времени последнего мониторинга, для этого определяем день недели и в зависимости от этого получаем новые данные
data = dt.datetime.isoweekday(dt.date.today())
date_now = dt.datetime.now().date().strftime('%d.%m.%Y')
if data == 1:
    last_monitoring = (dt.date.today() - dt.timedelta(days=3)).strftime('%Y-%m-%d')
else:
    last_monitoring = (dt.date.today() - dt.timedelta(days=2)).strftime('%Y-%m-%d')
    
df_dwh = df_dwh[df_dwh['Дата'] >= f'{last_monitoring}']

In [14]:
df_dwh.sample(5)

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
181583,3-05-133,2024-05-27 04:34:00,Веретельникова Ирина Евгеньевна,6212,Ch2-000825709,ТОВ1235371,TALANT 1,1,Футляр,9000135495130,ФУТЛЯРЫ,ФУТЛЯРЫ,ПРОЧЕЕ,14.530,NaN,0.000,NaN,0.000,NaN,160.000,0.000,160.000,ФУТЛЯР БРЕНД УНИВЕРС,160.000,+79136941585,NaN,160.000,0.000,160.000
192834,5-02-017,2024-05-27 11:25:00,Казакова Анастасия Дмитриевна,2705,Ch3-000777012,ТОВ1163703,ANTIBACTERIAL_WIPE,1,Аксессуар футляр,9000212169046,NaN,ДФРС,ЧС,0.000,NaN,0.000,NaN,0.000,NaN,10.000,0.000,10.000,САЛФЕТКА АНТИБАКТЕРИ,10.000,+79278190952,NaN,10.000,0.000,10.000
192413,3-03-063,2024-05-27 11:14:00,Зайчикова Полина Сергеевна,7229,Ch2-000827473,ТОВ1305840,КП 1,1,Футляр,9000212167783,NaN,ФУТЛЯРЫ,ФУТЛЯР,0.000,NaN,0.000,NaN,0.000,NaN,0.000,0.000,0.000,КОРОБОЧКА,0.000,(908)013-67-03,NaN,0.000,0.000,0.000
204666,4-01-173,2024-05-28 06:16:00,Тимофеева Диана Хасановна,2524,Ch2-000829016,ТОВ1172935,CR_BOX,1,Футляр,7651016136943,NaN,ФУТЛЯРЫ,ФУТЛЯР,0.000,NaN,0.000,NaN,0.000,NaN,0.000,0.000,0.000,КОРОБОЧКА 585*Золото,0.000,+79022774710,NaN,0.000,0.000,0.000
212941,6-05-004,2024-05-28 10:22:00,Погосян Милена Артуровна,894,Ch4-000522412,ТОВ1255454,50-78-1000-26986,1,Товар,7670003489800,СИ ПОДАРКИ КРОСС,СИ,ПОДВЕС ДЕКОРАТИВНЫЙ,0.000,СИ_НЕЛИКВИД_70%,0.516,925.000,0.000,"2,078,600,749,949.000","1,900.000",40.000,280.000,ПОДВЕС СЕРДЦЕ С БРИЛ,531.200,+79054619825,280.000,531.200,108.800,640.000


In [15]:
df_dwh['Товарная группа'].str.contains('ЦБ|БК').value_counts()

Товарная группа
False    21374
True      8437
Name: count, dtype: int64

In [16]:
df_dwh = df_dwh[(df_dwh['Товарное направление'].isin(['БК', 'ЦБ'])) & (df_dwh['Проба'] == 585.0)]

In [17]:
df_dwh.head()

,КБК,Дата,Продавец,Номер чека,ID продажи,Тов,Артикул поставщика,Количество,Номенклатура,Номер карты,Товарная группа,Товарное направление,Тип 1,Списание СберСпасибо,Группа наценки,Вес,Проба,Размер,ШК,Цена без скидки,Базовая скидка,Скидка в руб.,Название,Цена после скидки,Phone,Себестоимость,Сумма,Услуга,Общая сумма
180772,3-09-019,2024-05-27 00:04:00,Гусейнова Вероника Александровна,3565,Ch2-000825494,ТОВ0832965,КО 04-00,1,Товар,7651046360974,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,KVI,2.335,585.000,18.000,"2,078,608,760,853.000","42,966.000",70.000,"10,707.000",КОЛЬЦО ОБРУЧАЛЬНОЕ,"10,707.000",+79143609510,"8,420.040","10,707.000","2,193.000","12,900.000"
180774,3-09-019,2024-05-27 00:04:00,Гусейнова Вероника Александровна,3565,Ch2-000825494,ТОВ0832974,КО 04-00,1,Товар,7651046360974,БК КОЛЬЦА ОБРУЧ,БК,КОЛЬЦО ОБРУЧАЛЬНОЕ,0.000,KVI,2.717,585.000,22.000,"2,078,607,280,354.000","42,966.000",70.000,"11,505.230",КОЛЬЦО ОБРУЧАЛЬНОЕ,"11,505.230",+79143609510,"10,004.550","11,505.230","1,394.770","12,900.000"
180799,3-09-045,2024-05-27 01:25:00,Шумихина Вита Валентиновна,969,Ch2-000825499,ТОВ0881418,66.19_15,1,Товар,9000197333487,БК СЕРЬГИ,БК,СЕРЬГИ-КОНГО,0.000,ИФ.БК_ДЕШ,1.506,585.000,1.500,"2,078,496,163,095.000","26,566.000",29.000,"7,843.500",СЕРЬГИ-КОНГО,"7,843.500",+79245915114,"2,990.160","7,843.500","1,606.500","9,450.000"
180811,3-09-046,2024-05-27 01:40:00,Никифорова Ирина Евгеньевна,3786,Ch2-000825505,ТОВ1192085,20-01-0000-29515,1,Товар,9000212119096,БК КОЛЬЦА,БК,КОЛЬЦО,0.000,ИФ.БК_ДЕШ,1.242,585.000,16.500,"2,078,610,348,234.000","28,680.000",35.000,"7,760.500",КОЛЬЦО,"7,760.500",+79246833337,"4,114.820","7,760.500","1,589.500","9,350.000"
180817,3-09-018,2024-05-27 01:44:00,Дятлова Юлия Олеговна,7071,Ch2-000825512,ТОВ01260874,331-01-0060-37654,1,Товар,9000212119041,ЦБ ЦЕПИ,ЦБ,ЦЕПЬ,0.000,ЦБ_ПУСТОТЕЛЫЕ >3 ГРАММ,5.917,585.000,55.000,"2,078,610,115,977.000","104,475.000",20.000,"34,694.000",ЦЕПЬ БИСМАРК ПУСТ С,"34,694.000",+79241641575,"20,501.950","34,694.000","7,106.000","41,800.000"


In [18]:
df_dwh['Товарное направление'].unique()

array(['БК', 'ЦБ'], dtype=object)

In [19]:
df_dwh['Название'].isna().sum()

0

In [20]:
df_dwh['Название'] = df_dwh['Название'].fillna('золотое изделие')

In [22]:
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('каучук')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шнурок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('текстиль')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('хлопок')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('шелк')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('на нитке')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('кожа')))]
df_dwh = df_dwh[(~(df_dwh['Название'].str.lower().str.contains('жемчуг')))]

In [23]:
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'], format='%Y-%m-%d')
df_dwh['Дата'] = pd.to_datetime(df_dwh['Дата'].apply(lambda x: x.strftime("%Y-%m-%d")))

In [24]:
filtr_date = df_dwh['Дата'].max()

In [25]:
df_dwh['Дата'].min()

Timestamp('2024-05-27 00:00:00')

In [26]:
df_dwh = df_dwh.query("`Дата` <= @filtr_date")

In [27]:
df_dwh['Дата'].unique()

<DatetimeArray>
['2024-05-27 00:00:00', '2024-05-28 00:00:00']
Length: 2, dtype: datetime64[ns]

In [28]:
df_dwh['Товарное направление'].value_counts()

Товарное направление
БК    2940
ЦБ    2404
Name: count, dtype: int64

In [29]:
df_dwh.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5344 entries, 180772 to 224975
Data columns (total 29 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   КБК                   5344 non-null   object        
 1   Дата                  5344 non-null   datetime64[ns]
 2   Продавец              5344 non-null   object        
 3   Номер чека            5344 non-null   int64         
 4   ID продажи            5344 non-null   object        
 5   Тов                   5344 non-null   object        
 6   Артикул поставщика    5344 non-null   object        
 7   Количество            5344 non-null   int64         
 8   Номенклатура          5344 non-null   object        
 9   Номер карты           5140 non-null   object        
 10  Товарная группа       5344 non-null   object        
 11  Товарное направление  5344 non-null   object        
 12  Тип 1                 5344 non-null   object        
 13  Списание СберСпа

In [30]:
tn = df_dwh.groupby(by = 'Товарная группа', as_index= False)[['Себестоимость', 'Вес', 'Общая сумма']].agg('sum')

In [31]:
tn['Наценка продажи оффлайн'] =  tn['Общая сумма'] / tn['Себестоимость'] - 1

In [32]:
tn = tn[~tn['Товарная группа'].str.contains('ОПТ БК|БК ПЕЧАТКИ|ОПТ КОНГО|ОПТ ОБРУЧИ')]

In [33]:
tn['price_per_gramm'] = ((tn['Общая сумма'] / tn['Вес']).round(0))

In [34]:
tn['price_per_gramm'] = tn['price_per_gramm'].replace(',', '')
tn['price_per_gramm'] = tn['price_per_gramm'].astype('int')

In [35]:
tn

,Товарная группа,Себестоимость,Вес,Общая сумма,Наценка продажи оффлайн,price_per_gramm
0,БК КОЛЬЦА,"979,925.560",334.214,"2,201,170.000",1.246,6586
1,БК КОЛЬЦА ОБРУЧ,"8,595,048.140","2,955.989","18,045,055.270",1.099,6105
3,БК ПОДВЕС ДЕКОР,"617,353.550",197.075,"1,389,288.000",1.250,7050
4,БК ПОДВЕС КУЛЬТ,"2,875,881.650",940.755,"6,490,609.040",1.257,6899
5,БК СЕРЬГИ,"2,396,601.810",785.581,"5,392,223.750",1.250,6864
8,ЦБ БРАСЛЕТЫ,"9,491,666.860","2,854.096","18,771,628.580",0.978,6577
9,ЦБ ЦЕПИ,"18,603,445.260","5,768.846","38,966,465.830",1.095,6755


In [36]:
print('Средняя цена продажи оффлайн: ', (tn['Общая сумма'].sum() / tn['Вес'].sum()).round(0))

Средняя цена продажи оффлайн:  6595.0


In [38]:
tn.columns = ['tg', 'cost', 'weight', 'price', 'Наценка продажи оффлайн', 'price_per_gramm']

In [39]:
tn.to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sales_offline\offline_sales_{date_now}.xlsx', index = False)

In [40]:
tn = tn.drop(columns=['cost', 'weight', 'price', 'Наценка продажи оффлайн'])
tn.columns = ['tg', '585*Золотой оффлайн продажи']

In [33]:
pd.read_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx').merge(tn, how = 'left', on = 'tg').to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\sklv_sl_data\grouped_data\result_{date_now}.xlsx', index=False)